In [1]:
from pathlib import Path
import findspark
from pprint import pprint

findspark.init()

import pyspark
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import pandas as pd
import numpy as np

spark = SparkSession.builder.getOrCreate()

weather_paths = [str(p.resolve()) for p in Path("weather_data").glob("*/*")]

rows = spark.read.csv(weather_paths, header=True, inferSchema=True)
pd_df = rows.toPandas()
rows.printSchema()
rows.registerTempTable("weather")


23/04/01 13:26:47 WARN Utils: Your hostname, gadmin-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.3.155 instead (on interface ens160)
23/04/01 13:26:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/01 13:26:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


23/04/01 13:26:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


/home/gadmin/git/cloud_class/cloud_class_pyspark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


root
 |-- STATION: long (nullable = true)
 |-- DATE: timestamp (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ELEVATION: double (nullable = true)
 |-- NAME: string (nullable = true)
 |-- TEMP: double (nullable = true)
 |-- TEMP_ATTRIBUTES: double (nullable = true)
 |-- DEWP: double (nullable = true)
 |-- DEWP_ATTRIBUTES: double (nullable = true)
 |-- SLP: double (nullable = true)
 |-- SLP_ATTRIBUTES: double (nullable = true)
 |-- STP: double (nullable = true)
 |-- STP_ATTRIBUTES: double (nullable = true)
 |-- VISIB: double (nullable = true)
 |-- VISIB_ATTRIBUTES: double (nullable = true)
 |-- WDSP: double (nullable = true)
 |-- WDSP_ATTRIBUTES: double (nullable = true)
 |-- MXSPD: double (nullable = true)
 |-- GUST: double (nullable = true)
 |-- MAX: double (nullable = true)
 |-- MAX_ATTRIBUTES: string (nullable = true)
 |-- MIN: double (nullable = true)
 |-- MIN_ATTRIBUTES: string (nullable = true)
 |-- PRCP: double (nullable = 

/home/gadmin/git/cloud_class/cloud_class_pyspark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [2]:

years = spark.sql("select distinct year(date) as year from weather").orderBy('year').collect()
df_dict = {}
for y in years:
    df_dict[y.year] = rows.filter(F.year("date") == y.year)


```
Find the hottest day (column MAX) for each year, and provide the corresponding station code,
station name and the date (columns STATION, NAME, DATE).   > There should be 13 results.
```

In [3]:
max_temp = []
for key, df_weather in df_dict.items():
        max_filter = df_weather.select("date", "max", "station", "name").orderBy(F.desc("max"))
        max_temp.append(max_filter.collect()[0])
max_df = spark.createDataFrame(max_temp)
max_df.select("date", "max", "station", "name").orderBy(F.asc("date")).show()

+-------------------+------+-----------+--------------------+
|               date|   max|    station|                name|
+-------------------+------+-----------+--------------------+
|2010-08-15 00:00:00|  74.8|99407099999|DESTRUCTION IS. W...|
|2011-07-09 00:00:00|  87.8| 1046099999|       SORKJOSEN, NO|
|2012-07-05 00:00:00|  72.0| 1023099999|       BARDUFOSS, NO|
|2013-01-03 00:00:00|9999.9| 1001499999|      SORSTOKKEN, NO|
|2014-07-10 00:00:00|  89.6| 1023099999|       BARDUFOSS, NO|
|2015-07-30 00:00:00|  71.6| 1025099999|          TROMSO, NO|
|2016-03-10 00:00:00|9999.9| 1023199999|         DRAUGEN, NO|
|2017-06-09 00:00:00|  78.6| 1023099999|       BARDUFOSS, NO|
|2018-07-29 00:00:00|  84.2| 1025099999|          TROMSO, NO|
|2019-07-21 00:00:00|  78.8| 1023099999|       BARDUFOSS, NO|
|2020-06-22 00:00:00|  79.9| 1023099999|       BARDUFOSS, NO|
|2021-07-05 00:00:00|  88.3| 1065099999|        KARASJOK, NO|
|2022-07-01 00:00:00|  85.5| 2095099999|          PAJALA, SW|
+-------

```
Find the coldest day (column MIN) for the month of January across all years (2010 - 2022) ,
and provide the corresponding station code, station name and the date (columns STATION,
NAME, DATE). > There should be 1 result
```

In [4]:

min_temp = []
for key, df_weather in df_dict.items():
        min_filter = df_weather.select("date", "min", "station", "name").filter(F.month("date") == 1).orderBy(F.asc("min"))
        min_temp.append(min_filter.collect()[0])
min_df = spark.createDataFrame(min_temp)
min_df.select("date", "min", "station", "name").orderBy(F.asc("min")).show()


+-------------------+-----+----------+--------------------+
|               date|  min|   station|                name|
+-------------------+-----+----------+--------------------+
|2017-01-05 00:00:00|-28.3|1023099999|       BARDUFOSS, NO|
|2022-01-08 00:00:00|-28.1|2095099999|          PAJALA, SW|
|2021-01-13 00:00:00|-27.2|1065099999|        KARASJOK, NO|
|2011-01-30 00:00:00|-23.8|1008099999|        LONGYEAR, SV|
|2019-01-31 00:00:00|-22.0|1023099999|       BARDUFOSS, NO|
|2020-01-27 00:00:00|-19.3|1023099999|       BARDUFOSS, NO|
|2014-01-12 00:00:00|-18.9|1023099999|       BARDUFOSS, NO|
|2015-01-30 00:00:00| -8.7|1008099999|        LONGYEAR, SV|
|2012-01-30 00:00:00| -8.1|1023099999|       BARDUFOSS, NO|
|2018-01-03 00:00:00| -2.2|1008099999|        LONGYEAR, SV|
|2013-01-27 00:00:00| -1.3|1008099999|        LONGYEAR, SV|
|2010-01-29 00:00:00| -1.1|1052099999|HAMMERFEST AIRPOR...|
|2016-01-21 00:00:00| 10.4|1008099999|        LONGYEAR, SV|
+-------------------+-----+----------+--

```
Maximum and Minimum precipitation (column PRCP ) for the year 2015, and provide the
corresponding station code, station name and the date (columns STATION, NAME, DATE). > There should be 2 results.  Any max or min would do.  Just choose 1 or each.
```

In [5]:
max_min_max_filter = df_dict[2015].select("date", "prcp", "station", "name").orderBy(F.asc("prcp")).collect()[0]
max_min_min_filter = df_dict[2015].select("date", "prcp", "station", "name").orderBy(F.desc("prcp")).collect()[0]
max_min_df = spark.createDataFrame([max_min_max_filter, max_min_min_filter])
max_min_df.select("*").show()


+-------------------+-----+----------+------------+
|               date| prcp|   station|        name|
+-------------------+-----+----------+------------+
|2015-01-01 00:00:00|  0.0|1008099999|LONGYEAR, SV|
|2015-11-18 00:00:00|99.99|1008099999|LONGYEAR, SV|
+-------------------+-----+----------+------------+



```
Count percentage missing values for wind gust (column GUST) for the year 2019. > There should be 1 result
```

In [6]:
gust_total = df_dict[2019].select("date", "gust", "station", "name").orderBy(F.desc("gust")).count()
gust_missing = df_dict[2019].select("date", "gust", "station", "name").filter(F.col("gust") == "999.9").count()
print("Percentage of missing gusts is: " + format(gust_missing/gust_total, ".2%"))
#max_min_df.select("*").show()

Percentage of missing gusts is: 82.88%


```
Find the mean, median, mode and standard deviation of the Temperature (column TEMP) for
each month for the year 2020. > There should be 12 results, one for each month with 4 values for each result(row).
```

In [14]:
from pyspark.sql.types import FloatType
import statistics as st

def find_median(val_list):
    try:
        median = np.median(val_list)
        return round(float(median), 2)
    except Exception:
        return None
    
def find_mode(val_list):
    try:
        mode = st.mode(val_list)
        return round(float(mode), 2)
    except Exception:
        return None

df_dict[2020].registerTempTable("2020_weather")
months = spark.sql("select distinct month(date) as month from 2020_weather").orderBy('month').collect()
df_dict_2020 = {}
for m in months:
    df_dict_2020[m.month] = df_dict[2020].filter(F.month("date") == m.month)
df_dict_2020

median_finder = F.udf(find_median, FloatType())
mode_finder = F.udf(find_mode, FloatType())

for month, df_data in df_dict_2020.items():
    c = df_data.groupBy(F.month("date").alias("Month")).agg(F.collect_list("TEMP").alias("TEMP")
                                                            , F.mean("TEMP").alias("MEAN")
                                                            , F.stddev("TEMP").alias("STDDEV"))
    maths = c.collect()
    d = c.withColumn("MEAN", F.round("MEAN", 3)).withColumn("MEDIAN", median_finder("TEMP")).withColumn("MODE", mode_finder("TEMP")).withColumn("STDDEV", F.round("STDDEV", 3)).drop("TEMP")
    d.show()

+-----+------+------+------+----+
|Month|  MEAN|STDDEV|MEDIAN|MODE|
+-----+------+------+------+----+
|    1|15.897|12.805| 15.25| 5.7|
+-----+------+------+------+----+

+-----+------+------+------+----+
|Month|  MEAN|STDDEV|MEDIAN|MODE|
+-----+------+------+------+----+
|    2|13.359|13.092|  15.4| 8.7|
+-----+------+------+------+----+

+-----+------+------+------+----+
|Month|  MEAN|STDDEV|MEDIAN|MODE|
+-----+------+------+------+----+
|    3|14.653|15.785|  18.6|18.6|
+-----+------+------+------+----+

+-----+-----+------+------+----+
|Month| MEAN|STDDEV|MEDIAN|MODE|
+-----+-----+------+------+----+
|    4|23.33|13.022|  27.3|34.1|
+-----+-----+------+------+----+

+-----+------+------+------+----+
|Month|  MEAN|STDDEV|MEDIAN|MODE|
+-----+------+------+------+----+
|    5|36.219| 8.077| 36.05|37.0|
+-----+------+------+------+----+

+-----+-----+------+------+----+
|Month| MEAN|STDDEV|MEDIAN|MODE|
+-----+-----+------+------+----+
|    6|47.43| 8.877|  46.1|37.8|
+-----+-----+-----